In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

Mounted at /content/drive
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 220.6 MB 29 kB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
     |████████████████████████████████| 103 kB 26.6 MB/s 
     |████████████████████████████████| 109 kB 69.2 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
     |████████████████████████████████| 11.1 MB 60.1 MB/s 
     |████████████████████████████████| 51 kB 515 kB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 3.1 MB 32.7 MB/s 
     |████████████████████████████████| 546 kB 86.8 MB/s 
     |████████████████████████████████| 100 kB 12.6 MB/s 
     |████████████████████████████████| 90 kB 12.2 MB/s 
     |████████████████████████████████| 84 kB 4.5 MB/s 
     |████████████████████████████████| 121 kB 102.4 MB/s 
  Attempting uninstall: sphinx
    Found existing installa

In [2]:
%ls
%load_ext autoreload
%autoreload 2

drive/  sample_data/


In [3]:
%cd drive
%cd MyDrive
%cd SIG-VAE-GIN-and-fMRI
from sigvaemodel import *
from loss import *

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [14]:
%reload_ext autoreload
from sigvaemodel import *
from loss import loss as loss_function
from loss import get_rec

In [5]:
import numpy as np
import torch
import torch.nn as nn
import dgl.data
import networkx as nx
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score
import sigvaemodel
import loss
import time

CoraDataset=dgl.data.CoraGraphDataset()
graph=CoraDataset[0]
featureMatrix=graph.ndata['feat']
featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1])
print(featureMatrix.size())
print(featureMatrix)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
torch.Size([1, 2708, 1433])
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]])


In [6]:
def edges_to_lists(headMatrix, tailMatrix):
    results=[]
    assert len(headMatrix)==len(tailMatrix)
    hlength=len(headMatrix)
    for i in range(hlength):
        results.append([headMatrix[i], tailMatrix[i]])
    return results

In [7]:
def preprocess(graph):
    N=graph.num_nodes()
    u, v=graph.edges()
    testidx=np.arange(graph.number_of_edges())
    testidx=np.random.permutation(testidx)
    testSize=int(len(testidx)/10)
    train_size=graph.number_of_edges()-testSize
    test_pos_u, test_pos_v=u[testidx[:testSize]], v[testidx[:testSize]]
    train_pos_u, train_pos_v = u[testidx[testSize:]], v[testidx[testSize:]]

    
    adj=sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
    adj_neg=1-adj.todense()-np.eye(graph.number_of_nodes())
    adj=torch.Tensor(adj.todense())
    neg_u, neg_v=np.where(adj_neg!=0)
    #print(len(neg_u))
    #print(len(neg_v))
    neg_testidx=np.random.choice(len(neg_u), graph.number_of_edges())
    test_neg_u, test_neg_v=neg_u[neg_testidx[:testSize]], neg_v[neg_testidx[:testSize]]
    train_neg_u, train_neg_v=neg_u[neg_testidx[testSize:]], neg_v[neg_testidx[testSize:]]
    #print(test_neg_u)
    #print(test_neg_v)
    adj_train=sp.coo_matrix((np.ones(len(train_pos_u)), (train_pos_u.numpy(), train_pos_v.numpy())),shape=(N, N))
    
    adj_train=adj_train-sp.dia_matrix((adj_train.diagonal()[np.newaxis, :], [0]), shape=adj_train.shape)
    adj_train=torch.Tensor(adj_train.todense())
    
    adj_test=sp.coo_matrix((np.ones(len(test_pos_u)), (test_pos_u.numpy(), test_pos_v.numpy())))
    adj_test=adj_test-sp.dia_matrix((adj_test.diagonal()[np.newaxis, :], [0]), shape=adj_test.shape)
    adj_test=torch.Tensor(adj_test.todense())
    train_graph=dgl.remove_edges(graph, testidx[:testSize])
    train_pos=edges_to_lists(train_pos_u, train_pos_v)
    test_pos=edges_to_lists(test_pos_u, test_pos_v)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)

    featureMatrix=graph.ndata['feat']
    featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1])

        
    return adj, adj_train, adj_test, featureMatrix, train_graph, train_pos, test_pos, train_neg, test_neg


In [8]:
def step(model, adj, feat, epoch, optimizer, device):
    global loss
    if optimizer is None:
        model.eval()
    else:
        model.train()
    
    generated_prob, mu, sigma, latent_representation, Z, epsilon=model.forward(adj.to(device), feat.to(device))
    loss_rec, loss_prior, loss_post=loss_function(generated_prob, adj, mu, sigma, Z, epsilon, latent_representation, device=device)
    loss=loss_rec+loss_prior+loss_post
    
    WarmingUP=np.min([epoch/30, 1])
    reg=(loss_post-loss_prior) * WarmingUP / (adj.size(0)**2)
    loss_train=loss_rec+WarmingUP*reg
    
    if optimizer is not None:
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
    return latent_representation, generated_prob, loss_train, Z

In [9]:
def get_auc(embedding, pos_edges, neg_edges):
    pos_array=np.array(pos_edges) #(2, #positive edges)
    neg_array=np.array(neg_edges) #(2, #negative edges)
    emb_pos_head=embedding[:, pos_array[0], :]
    emb_pos_tail=embedding[:, pos_array[1], :]
    preds_pos=np.einsum('abc, abc->ab', emb_pos_head, emb_pos_tail)
    
    emb_neg_head=embedding[:, neg_array[0], :]
    emb_neg_tail=embedding[:, neg_array[1], :]
    preds_neg=np.einsum('abc, abc->ab', emb_neg_head, emb_neg_tail)
    
    preds=np.concatenate([preds_pos, preds_neg], axis=0)
    labels=np.concatenate([np.ones(preds_pos.shape[-1]), np.zeros(preds_neg.shape[-1])])
    
    roc=torch.Tensor([roc_auc_score(labels, pred.flatten()) for pred in np.split(preds, embedding.shape[0])]).mean()
    
    return roc

In [15]:
if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'

adj, adj_train, adj_test, featureMatrix, train_graph, train_pos, test_pos, train_neg, test_neg=preprocess(graph)
adj_label=adj_train+torch.eye(adj_train.shape[0])
adj_label=torch.FloatTensor(adj_label)
adj=torch.Tensor(adj).to(device)
adj_test=torch.Tensor(adj_test).to(device)
featureMatrix=torch.Tensor(featureMatrix).to(device)
adj_train=torch.Tensor(adj_train).to(device)

print(torch.count_nonzero(adj_train))
print(torch.count_nonzero(adj_test))

model=SIGVAE_GIN(Lu=1, 
                Lmu=1, 
                Lsigma=1, 
                input_dim=featureMatrix.shape[2], 
                output_dim_u=[32], 
                output_dim_mu=[16], 
                output_dim_sigma=[16],
                K=15,
                J=20,
                device=device,
                noise_dim=64,
                activation=torch.nn.functional.relu,
                dropout=0)
optimizer=torch.optim.Adam(model.parameters(), lr=0.0005)
epoch=100
    
for epc in range(epoch):
    t=time.time()
    model.train()
    optimizer.zero_grad()
    

    loss_accumulate=0
    latent_representation, generated_prob, loss_train, z_proc=step(model, adj_train, featureMatrix, epc, optimizer, device=device)
    loss_accumulate+=loss_train
    
    z_proc=z_proc.detach().cpu().numpy()
    roc=get_auc(z_proc, train_pos, train_neg)
    #print(loss_train.item())
    #print(roc)
    print("Epoch : ", '%d' %(epc+1), "loss=", "%f" %(loss_train.item()),
          "time=", "%f" %(time.time()-t), "roc=", "%f" %(roc))
    
    if (epc+1) % 10 == 0:
            model.eval()
            generated_prob, mu, sigma, latent_representation, z_proc, epsilon=model(adj_train, featureMatrix)
            z_proc = z_proc.detach().cpu().numpy()
            roc = get_auc(z_proc, test_pos, test_neg)
            rslt = "Test ROC score: %f" %(roc)
            print("\n", rslt, "\n")
            del mu
            del sigma
    del latent_representation
    del generated_prob
    del z_proc
    del roc
    
    torch.cuda.empty_cache()
    #latent_representation, generated_prob, loss_train=step(model, adj_train, epoch, optimizer )

tensor(9501, device='cuda:0')
tensor(1055, device='cuda:0')
Epoch :  1 loss= 385.132538 time= 7.877650 roc= 0.437500
Epoch :  2 loss= 383.719971 time= 7.107721 roc= 0.612500
Epoch :  3 loss= 383.182678 time= 7.164263 roc= 0.487500
Epoch :  4 loss= 382.935333 time= 7.189595 roc= 0.550000
Epoch :  5 loss= 382.753662 time= 7.180806 roc= 0.425000
Epoch :  6 loss= 382.674866 time= 7.165407 roc= 0.525000
Epoch :  7 loss= 382.632111 time= 7.043534 roc= 0.487500
Epoch :  8 loss= 382.653137 time= 7.029943 roc= 0.525000
Epoch :  9 loss= 382.603333 time= 7.114284 roc= 0.525000
Epoch :  10 loss= 382.609833 time= 7.083760 roc= 0.525000

 Test ROC score: 0.575000 

Epoch :  11 loss= 382.566010 time= 7.146768 roc= 0.550000
Epoch :  12 loss= 382.546844 time= 7.138631 roc= 0.525000
Epoch :  13 loss= 382.517059 time= 7.139636 roc= 0.525000
Epoch :  14 loss= 382.514587 time= 7.186982 roc= 0.512500
Epoch :  15 loss= 382.448639 time= 7.168781 roc= 0.625000
Epoch :  16 loss= 382.408966 time= 7.069801 roc= 0

KeyboardInterrupt: ignored

In [12]:
!nvidia-smi

Mon May 23 04:04:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |  11124MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------